In [1]:
import findspark
findspark.init()

In [2]:
import json

In [3]:
from pyspark import SparkConf, SparkContext, StorageLevel
from pyspark.sql import SparkSession

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [5]:
file_path = "file:///Users/markduan/duan/USC_course/USC_APDS/INF553/project/data/train_review.json"

In [6]:
conf = SparkConf() \
    .setAppName("task") \
    .setMaster("local[*]") \
    .set("spark.driver.memory","4g")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

In [7]:
spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()

In [8]:
def transformToRow(r, b_t, u_t):
    rel = {}
    rel['uid'] = u_t.index(r['user_id'])
    rel['bid'] = b_t.index(r['business_id'])
    rel['stars'] = r['stars']
    return rel

In [9]:
b_table = sc.textFile(file_path) \
    .map(json.loads) \
    .map(lambda x: x['business_id']) \
    .distinct() \
    .collect()

In [10]:
u_table = sc.textFile(file_path) \
    .map(json.loads) \
    .map(lambda x: x['user_id']) \
    .distinct() \
    .collect()

In [11]:
stars_df = sc.textFile(file_path) \
    .map(json.loads) \
    .map(lambda x: Row(**transformToRow(x, b_table, u_table))) \
    .toDF()

In [12]:
stars_df.show()

+-----+-----+-----+
|  bid|stars|  uid|
+-----+-----+-----+
| 8030|  4.0|28254|
|12652|  5.0|49434|
| 6543|  5.0|10585|
| 5028|  5.0|35239|
| 7062|  5.0|56400|
| 8501|  1.0|28255|
| 3537|  1.0|35240|
|10029|  4.0|14088|
|    0|  5.0|45879|
|10030|  3.0|77587|
|11085|  4.0|21130|
| 3037|  4.0|84678|
| 9056|  5.0|    0|
|10031|  5.0|49435|
| 4063|  4.0|31708|
| 2505|  4.0|52904|
|10583|  4.0|63599|
|10584|  4.0|21131|
|10032|  4.0|42316|
| 2506|  2.0|14089|
+-----+-----+-----+
only showing top 20 rows



In [22]:
# training, testing = stars_df.randomSplit([0.8, 0.2])

In [13]:
model = ALS(rank=1, maxIter=15, regParam=0.01, userCol="uid", itemCol='bid', ratingCol='stars', nonnegative=True)

In [33]:
# model.setColdStartStrategy('drop')

ALS_42d6a2b63913c4fbdee2

In [14]:
model_exp = model.fit(stars_df)

In [15]:
model_file = "file:///Users/markduan/duan/USC_course/USC_APDS/INF553/project/model/ml_als.model"

In [16]:
model_exp.save(model_file)

Py4JJavaError: An error occurred while calling o205.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:224)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc

In [35]:
prediction_exp = model_exp.transform(testing)

In [36]:
prediction_exp.show()

+---+-----+-----+-----------+
|bid|stars|  uid| prediction|
+---+-----+-----+-----------+
|148|  1.0| 5025|   5.977809|
|148|  5.0|64159|  4.3979387|
|148|  5.0| 3930|   4.913999|
|148|  4.0|29289|  4.1816072|
|148|  3.0|22052|  3.2904627|
|148|  4.0|81342|   5.623118|
|148|  4.0|87283|0.012917876|
|148|  5.0|81371|  11.550993|
|148|  1.0|85057|  4.2061324|
|463|  5.0|20085|  0.5430939|
|463|  4.0|60990|    7.71448|
|471|  5.0|13047| -0.8829052|
|471|  5.0|55110| -0.2319829|
|496|  4.0|17959|  2.5830822|
|496|  5.0| 9695|  1.2027736|
|496|  5.0|71901|0.052501082|
|496|  5.0| 7341| 0.17471725|
|496|  2.0|67571|  0.6460347|
|496|  5.0|68261|  6.5621157|
|496|  5.0|67914| 0.49163648|
+---+-----+-----+-----------+
only showing top 20 rows



In [37]:
evaluator = RegressionEvaluator().setMetricName('rmse').setLabelCol('stars').setPredictionCol('prediction')

In [38]:
rmse_exp.evaluate(prediction_exp)

2.8827493854619655

In [17]:
l1 = ['s','t','a']

In [18]:
l1.index('t')

1